# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 12 2023 8:30AM,254852,10,0086294196,ISDIN Corporation,Released
1,Jan 12 2023 8:30AM,254852,10,0086294197,ISDIN Corporation,Released
2,Jan 12 2023 8:30AM,254852,10,0086294195,ISDIN Corporation,Released
3,Jan 12 2023 8:30AM,254852,10,0086294198,ISDIN Corporation,Released
4,Jan 12 2023 8:30AM,254853,10,SO94491,"Senseonics, Incorporated",Released
5,Jan 12 2023 8:30AM,254847,10,9019426,"Akron BioProducts, LLC",Released
6,Jan 12 2023 8:30AM,254852,10,0086294189,ISDIN Corporation,Completed
7,Jan 12 2023 8:09AM,254869,16,SHL-9022548,"SHL Pharma, LLC",Released
8,Jan 11 2023 5:44PM,254867,19,ADV80010270,"AdvaGen Pharma, Ltd",Released
9,Jan 11 2023 5:44PM,254867,19,ADV80010271,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,254864,Released,46
46,254865,Released,94
47,254866,Released,1
48,254867,Released,17
49,254869,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254864,NaN,NaN,46.0
254865,NaN,NaN,94.0
254866,NaN,NaN,1.0
254867,NaN,NaN,17.0
254869,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254736,7.0,0.0,3.0
254737,9.0,0.0,1.0
254744,0.0,1.0,0.0
254747,0.0,0.0,1.0
254753,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254736,7,0,3
254737,9,0,1
254744,0,1,0
254747,0,0,1
254753,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254736,7,0,3
1,254737,9,0,1
2,254744,0,1,0
3,254747,0,0,1
4,254753,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254736,7,,3
1,254737,9,,1
2,254744,,1,
3,254747,,,1
4,254753,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 12 2023 8:30AM,254852,10,ISDIN Corporation
4,Jan 12 2023 8:30AM,254853,10,"Senseonics, Incorporated"
5,Jan 12 2023 8:30AM,254847,10,"Akron BioProducts, LLC"
7,Jan 12 2023 8:09AM,254869,16,"SHL Pharma, LLC"
8,Jan 11 2023 5:44PM,254867,19,"AdvaGen Pharma, Ltd"
25,Jan 11 2023 5:37PM,254865,15,"Person & Covey, Inc."
119,Jan 11 2023 5:36PM,254866,15,"Alliance Pharma, Inc."
120,Jan 11 2023 5:36PM,254864,19,"AdvaGen Pharma, Ltd"
166,Jan 11 2023 4:45PM,254862,21,"NBTY Global, Inc."
167,Jan 11 2023 4:38PM,254858,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 12 2023 8:30AM,254852,10,ISDIN Corporation,1,,4
1,Jan 12 2023 8:30AM,254853,10,"Senseonics, Incorporated",,,1
2,Jan 12 2023 8:30AM,254847,10,"Akron BioProducts, LLC",,,1
3,Jan 12 2023 8:09AM,254869,16,"SHL Pharma, LLC",,,1
4,Jan 11 2023 5:44PM,254867,19,"AdvaGen Pharma, Ltd",,,17
5,Jan 11 2023 5:37PM,254865,15,"Person & Covey, Inc.",,,94
6,Jan 11 2023 5:36PM,254866,15,"Alliance Pharma, Inc.",,,1
7,Jan 11 2023 5:36PM,254864,19,"AdvaGen Pharma, Ltd",,,46
8,Jan 11 2023 4:45PM,254862,21,"NBTY Global, Inc.",,,1
9,Jan 11 2023 4:38PM,254858,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 8:30AM,254852,10,ISDIN Corporation,4,,1
1,Jan 12 2023 8:30AM,254853,10,"Senseonics, Incorporated",1,,
2,Jan 12 2023 8:30AM,254847,10,"Akron BioProducts, LLC",1,,
3,Jan 12 2023 8:09AM,254869,16,"SHL Pharma, LLC",1,,
4,Jan 11 2023 5:44PM,254867,19,"AdvaGen Pharma, Ltd",17,,
5,Jan 11 2023 5:37PM,254865,15,"Person & Covey, Inc.",94,,
6,Jan 11 2023 5:36PM,254866,15,"Alliance Pharma, Inc.",1,,
7,Jan 11 2023 5:36PM,254864,19,"AdvaGen Pharma, Ltd",46,,
8,Jan 11 2023 4:45PM,254862,21,"NBTY Global, Inc.",1,,
9,Jan 11 2023 4:38PM,254858,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 8:30AM,254852,10,ISDIN Corporation,4,,1
1,Jan 12 2023 8:30AM,254853,10,"Senseonics, Incorporated",1,,
2,Jan 12 2023 8:30AM,254847,10,"Akron BioProducts, LLC",1,,
3,Jan 12 2023 8:09AM,254869,16,"SHL Pharma, LLC",1,,
4,Jan 11 2023 5:44PM,254867,19,"AdvaGen Pharma, Ltd",17,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 8:30AM,254852,10,ISDIN Corporation,4.0,NaN,1.0
1,Jan 12 2023 8:30AM,254853,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Jan 12 2023 8:30AM,254847,10,"Akron BioProducts, LLC",1.0,NaN,NaN
3,Jan 12 2023 8:09AM,254869,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,Jan 11 2023 5:44PM,254867,19,"AdvaGen Pharma, Ltd",17.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 8:30AM,254852,10,ISDIN Corporation,4.0,0.0,1.0
1,Jan 12 2023 8:30AM,254853,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Jan 12 2023 8:30AM,254847,10,"Akron BioProducts, LLC",1.0,0.0,0.0
3,Jan 12 2023 8:09AM,254869,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,Jan 11 2023 5:44PM,254867,19,"AdvaGen Pharma, Ltd",17.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4076697,37.0,0.0,31.0
15,764545,115.0,0.0,33.0
16,764360,2.0,1.0,0.0
19,1529114,78.0,0.0,0.0
21,4586865,18.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4076697,37.0,0.0,31.0
1,15,764545,115.0,0.0,33.0
2,16,764360,2.0,1.0,0.0
3,19,1529114,78.0,0.0,0.0
4,21,4586865,18.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,37.0,0.0,31.0
1,15,115.0,0.0,33.0
2,16,2.0,1.0,0.0
3,19,78.0,0.0,0.0
4,21,18.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,15,Released,115.0
2,16,Released,2.0
3,19,Released,78.0
4,21,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,31.0,33.0,0.0,0.0,0.0
Executing,0.0,0.0,1.0,0.0,0.0
Released,37.0,115.0,2.0,78.0,18.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,31.0,33.0,0.0,0.0,0.0
1,Executing,0.0,0.0,1.0,0.0,0.0
2,Released,37.0,115.0,2.0,78.0,18.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,31.0,33.0,0.0,0.0,0.0
1,Executing,0.0,0.0,1.0,0.0,0.0
2,Released,37.0,115.0,2.0,78.0,18.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()